In [ ]:
import os
import sys

In [ ]:
# Theodore's paths (do not run this cell)
os.environ["PYLIB"] = os.environ["SPARK_HOME"] + "/python/lib"
#os.environ['PYSPARK_SUBMIT_ARGS'] = f'--packages org.apache.spark:spark-sql-kafka-0-10_2.11:2.4.1,org.apache.spark:spark-streaming-kafka-0-8_2.11:2.2.0 --jars spark-sql-kafka-0-10_2.11-2.4.1.jar pyspark-shell'
sys.path.insert(0, os.environ["PYLIB"] +"/py4j-0.10.7-src.zip")
sys.path.insert(0, os.environ["PYLIB"] +"/pyspark.zip")
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-8_2.11:2.4.4 pyspark-shell'

In [ ]:

os.environ["SPARK_HOME"]="/home/user1/trainig_material/spark-2.4.4-bin-hadoop2.7"
os.environ["PYLIB"] = os.environ["SPARK_HOME"] + "/python/lib"
#os.environ["HIVE_HOME"] ="/usr/lib/hive"
#os.environ["PATH"] =os.environ["HIVE_HOME"]+"/bin"
#os.environ["JAVA_HOME"]="/usr/java/jdk1.7.0_67-cloudera"
# In below two lines, use /usr/bin/python2.7 if you want to use Python 2
#os.environ["PYSPARK_PYTHON"] = "/home/cloudera/anaconda3/bin/python3.7" 
#os.environ["PYSPARK_DRIVER_PYTHON"] = "/home/cloudera/anaconda3/bin/python3.7"
sys.path.insert(0, os.environ["PYLIB"] +"/py4j-0.10.7-src.zip")
sys.path.insert(0, os.environ["PYLIB"] +"/pyspark.zip")


os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-8_2.11:2.4.4 pyspark-shell'

In [ ]:

!pyspark --version

In [ ]:
# Spark
from pyspark import SparkContext, SparkConf
# Spark Streaming
from pyspark.streaming import StreamingContext
# Kafka
#from pyspark.streaming.kafka import KafkaUtils
# json parsing
from pyspark.sql import Row,SQLContext
from pyspark import SparkConf,SparkContext
from pyspark.sql import SQLContext,HiveContext

In [ ]:
sc = SparkContext("local", "twitter")
sc.setLogLevel("WARN")

In [ ]:
sc.stop()

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.enableHiveSupport().getOrCreate()

In [ ]:
spark.sql("show databases").show()

In [ ]:
import pandas as pd
from datetime import datetime, timedelta
import json

import tweepy
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
from textblob import TextBlob

from kafka import KafkaClient
from kafka import KafkaProducer
from kafka import KafkaConsumer
from kafka import TopicPartition
from kafka.errors import KafkaError

In [ ]:
consumer = ''

In [ ]:
# not utilizing a structured streaming environment, this will allow us to save tweets in consumer 
#nd not neet to run producer and streaming simultaneosly avoiding error:420
consumer = KafkaConsumer('final_twitter', 
                         group_id='1', # Using group so that it doesn't re-read messages
                         auto_offset_reset='earliest', 
                         enable_auto_commit=True, 
                         bootstrap_servers=['localhost:9092'], 
                         value_deserializer=lambda m: json.loads(m))
# Create TopicPartition object so we can keep track of offset and know when to exit consumer loop later
tp = TopicPartition('final_twitter',0)

In [ ]:
rows = [] # Put in separate cell so we don't clear rows every time we run next cell

In [ ]:
date_format = '%a %b %d %H:%M:%S %Y' # To convert created_at to Datetime object

#list of dicts, this will save tweets from consumer
for msg in consumer:
    # Obtain the last offset value, so that we know when to exit loop
    current_position = consumer.position(tp) # Store current offset pos so we can seek to it after
    consumer.seek_to_end(tp)
    last_offset = consumer.position(tp)
    
    consumer.seek(tp, current_position) # Seek back to where we were
    print("\nCurrent offset:", msg.offset)
    print("Last offset:", last_offset)
    
    # Comment this out to continue even if consumer catches up to producer
    if msg.offset >= last_offset - 1: 
        break
    
    # this converts consumer to type dict
    row = msg.value
    # pull the hashtags
    if len(row['entities']['hashtags']) > 0:
        #hashtag_text = [row['entities']['hashtags'][0]['text'] # Get text string of first hashtag
        hashtag_text = [row['entities']['hashtags'][i]['text'].lower() for i, tag in enumerate(row['entities']['hashtags'])]
    else:
        #continue # Commented this out since if we continue, the next line wouldn't execute and we wouldn't store the tweet
        hashtag_text = []
    # create a row and add it to the list, we only need Tweet Id, User Id, Text, hashtags, and event time
    row = Row(TweetID=row['id'], 
              Text=row['text'], 
              UserID=row['user']['id'], 
              Hashtags=hashtag_text,
              #Event_time=row['created_at']
              Event_time=datetime.strptime(row['created_at'].replace('+0000 ', ''), date_format)
    )
    rows.append(row)
    print(row)

In [ ]:
# sanity check
print(rows[:10]) # First 10 rows
print('\nNumber of tweets collected: {}'.format(len(rows)))

In [ ]:
# convert the list of dicts to a spark dataframe as per the requirements
df = spark.createDataFrame(rows)

In [ ]:
df.show()

## Number of tweets in last 30 second window

In [ ]:
df.createOrReplaceTempView("tweet_df")

In [ ]:
# Probably won't capture anything unless run right away, since it uses current time instead of most recent tweet
# Maybe better to use below method
spark.sql("""
    SELECT count(*) as num_tweets_last_30
    FROM tweet_df
    WHERE Event_time between (current_timestamp() - INTERVAL 30 SECOND ) 
                                AND current_timestamp()
""").show()

In [ ]:
# Get the most recent tweet event_time captured, store as datetime
# We can use this for all other queries to compare against most recent tweet (instead of current time, since it might be delayed)
most_recent_tweet = spark.sql("""
    SELECT Event_time
    FROM tweet_df
    ORDER BY Event_time 
    DESC LIMIT 1""").collect()[0].Event_time

In [ ]:
print('Datetime of most recently captured tweet is: {}'.format(most_recent_tweet)) 

In [ ]:

query = """
    SELECT COUNT(*) as Num_Tweets_Last_30_From_Newest_Tweet
    FROM tweet_df
    WHERE Event_time between ('{}' - INTERVAL 30 SECOND ) 
                                AND '{}'""".format(most_recent_tweet, most_recent_tweet)
print(query)
num_tweets_last_30 = spark.sql(query)
num_tweets_last_30.show()

## Num active users last 30 seconds

In [ ]:
# Actual last 30 seconds from current time
query = """
    SELECT count(distinct(UserID)) as Num_Active_Users_Last_30_From_Current
    FROM tweet_df
    WHERE Event_time between (current_timestamp() - INTERVAL 30 SECOND ) 
                                AND current_timestamp()""".format(most_recent_tweet)
spark.sql(query).show()


In [ ]:
# 30 seconds prior to most recently received tweet
query = """
    SELECT count(distinct(UserID)) as Num_Active_Users_Last_30_From_Newest_Tweet
    FROM tweet_df
    WHERE Event_time between ('{}' - INTERVAL 30 SECOND ) 
                                AND '{}'""".format(most_recent_tweet, most_recent_tweet)
print(query)
spark.sql(query).show()

## Most used hashtag in last 30 seconds

In [ ]:
# key=lambda a: a[1] Means use the second value in the (word, num_occurrences) tuple as comparison key for max
query = """
    SELECT Hashtags
    FROM tweet_df
    WHERE Event_time between ('{}' - INTERVAL 30 SECOND ) 
                                AND '{}'""".format(most_recent_tweet, most_recent_tweet)
df_last_30 = spark.sql(query)
frequent_hashtag_tup_last_30 = max(df_last_30.select(['Hashtags']).rdd.flatMap(lambda a: a.Hashtags).countByValue().items(), key=lambda a: a[1])
print('Most frequently occuring hashtag in last 30 seconds: {}'.format(frequent_hashtag_tup_last_30[0]))
print('Num occurrences: {}'.format(frequent_hashtag_tup_last_30[1]))

## Most used hashtag in complete time of analysis

In [ ]:
# key=lambda a: a[1] Means use the second value in the (word, num_occurrences) tuple as comparison key for max
frequent_hashtag_tup_complete = max(df.select(['Hashtags']).rdd.flatMap(lambda a: a.Hashtags).countByValue().items(), key=lambda a: a[1])
print('Most frequently occuring hashtag in complete time of analysis: {}'.format(frequent_hashtag_tup_complete[0]))
print('Num occurrences: {}'.format(frequent_hashtag_tup_complete[1]))

## Moving average of a given Hashtag every 1 minute

In [ ]:
from pyspark.sql.window import Window
from pyspark.sql import functions as func
from pyspark.sql.functions import explode
# I basically followed this tutorial: 
# https://www.linkedin.com/pulse/time-series-moving-average-apache-pyspark-laurent-weichberger/


In [ ]:
# Create a "seven day" Window from seven days previous to the current day (row), using previous casting of timestamp to long (number of seconds).

# remember a start value of "-1" means one off before the current row, and we are taking the timestamp as a long and comparing it to the rangeBetween amount of time.

# Remember an end value of 0 is the current row.
# We use 6 days because rangeBetween is inclusive of start & end values.
# Thank you Dr. Kevin Maguire for the bug fix!
#windowSpec = Window.orderBy(func.col("Event_time").cast('long')).rangeBetween(2, 0) # Using 3 seconds for now. Set to 59 for 1 minute
windowSpec = Window.orderBy(func.col("Event_time").cast('long')).rangeBetween(2, 0) # Using 3 seconds for now. Set to 59 for 1 minute

In [ ]:
# Explode the dataframe so that each element in each Hashtag array has its own row, 
# and so that the Hashtags column now only contains one hashtag string
df_exploded = df.withColumn('Hashtags', explode('Hashtags'))
df_exploded.show()

In [ ]:
# Now create a new column which has the count of 

In [ ]:
# Filter DF such that it only contains rows with our hashtag of interest
df2 = df_exploded.filter(df_exploded.Hashtags == frequent_hashtag_tup_complete[0])

In [ ]:
df2.show()

In [ ]:
from pyspark.sql.functions import lit
df2 = df2.withColumn('count', lit(1.0)) # Simply add a new column and set each val to 1 (used for moving avg later)
df2.show()

In [ ]:
# Edit: Not sure if we need this
# We do a wordcount on Event_time, because at this point each Event_time row is one occurance for the most frequent hashtag
from pyspark.sql.types import DoubleType
import pyspark.sql.functions as f
df3 = df2 \
        .groupBy('Event_time') \
        .count() \
        .sort('Event_time', ascending=True)
df3.show()

In [ ]:
# SparkSQL method - not sure if this is correct
df3.createOrReplaceTempView("df3")
spark.sql(
    """SELECT *, mean(count) OVER (
        ORDER BY CAST(Event_time AS timestamp) 
        RANGE BETWEEN INTERVAL 60 SECONDS PRECEDING AND CURRENT ROW
     ) AS Moving_Average FROM df3""").show()

In [ ]:
# Tried other method but not sure why this returns null. Not needed since above works

df4 = df3.withColumn('Moving Average', func.avg("count").over(windowSpec)) 
print('Showing moving average for hashtag: {}'.format(frequent_hashtag_tup_complete[0]))
df4.show()

## Write df to hive

In [ ]:
# Add timestamp to this record in case we want to make sense of the data in the hive table later
# Also, reorder columns so timestamp is on left
table_to_write = num_tweets_last_30.withColumn('End_of_Capturing_Window', lit(most_recent_tweet))
table_to_write = table_to_write.select('End_of_Capturing_Window', 'Num_Tweets_Last_30_From_Newest_Tweet')
table_to_write.show()

In [ ]:

# Create db and table
spark.sql("""
    CREATE DATABASE twitter_analysis
""")

In [ ]:
spark.sql("""
    USE twitter_analysis
""").show()

In [ ]:
spark.sql("""
    CREATE TABLE IF NOT EXISTS tweets_last_30(
        End_of_Capturing_Window String, 
        Num_Tweets_Last_30_From_Newest_Tweet String
    )
""").show()

In [ ]:
table_to_write.write.mode("append").format('hive').saveAsTable("twitter_analysis.tweets_last_30")

In [ ]:
spark.sql("""
    SELECT *
    FROM tweets_last_30
""").show()